# UN Capital Population Data
### by Gopal Katyarmal

## Version 1 - raw work
##### Date : 10th March 2023

## Version 2 - Proper labelling and Headings
##### Date : 27th March 2023

## Introduction:
Here we are analysing the data of caital cities over period of a decade provided by UN. For first I have removed a lot of unnecessary columns and remove all rows which don't have Capital City mentioned. Biggest part here is of transformation on rows. Converting rows with given year for a city to columns of same year for same city . Later part is of Basic visualization of top few entries and some conlcusion drawn from it.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import requests
res = pd.read_csv("https://data.un.org/_Docs/SYB/CSV/SYB61_253_Population%20Growth%20Rates%20in%20Urban%20areas%20and%20Capital%20cities.csv", encoding = 'cp437',engine = 'python')


### Adjusting the names of index.

In [ ]:
res.iloc[0] = res.iloc[0].fillna( 'Name')
print(res.iloc[0])
res.columns = res.iloc[0]
res.drop(0)

In [ ]:
res.describe()
# res["Capital City"].unique()

### Droping all rows whose Capital City is not given as this field is the center of this data

In [ ]:
print(res.shape)
res = res.dropna(subset = ["Capital City"])
print(res.shape)

In [ ]:
res.describe()

### Transforming data 1 : Converting all rows of 'Capital city population (as a percentage of total population)' to a column for same city . 

In [ ]:
res.Series.value_counts()

In [ ]:
percent_pop_total = res[res["Series"] == "Capital city population (as a percentage of total population)"][["Year", "Capital City", "Value"]]
res = res[res["Series"] == "Capital city population (thousands)" ]
print(res.shape)

In [ ]:
percent_pop_total = percent_pop_total.reset_index()
percent_pop_total["index"] = percent_pop_total["index"] - 1
percent_pop_total.rename(columns = {"Value":"Percent_value", "index":"id"}, inplace = True)
res = res.reset_index()
res.rename(columns = {'index':'id'}, inplace = True)
print(percent_pop_total.columns)
print(res.columns)


In [ ]:
res = res.merge(percent_pop_total[['id', "Percent_value"]], how= 'left' )


### Droping other useless columns 

In [ ]:
res.drop("id", axis =1, inplace = True)
res.drop("Region/Country/Area", axis =1, inplace = True)
res.drop("Series", axis =1, inplace = True)
res.drop("Capital City footnote", axis = 1, inplace = True)
res.drop("Source", axis =1, inplace = True)
res.drop("Footnotes", axis =1, inplace = True)
res.Name = res["Capital City"] + ", "+res.Name
res.drop("Capital City", inplace =True, axis =1)
res.head()

In [ ]:
res.Name.unique().size
res.Year.dtype
res[res["Year"]=="2018"].Percent_value.unique()

### Data Transformation 2: Converting rows with given years for a particular city to columns of same year and same city

In [ ]:
data_01 = res[res.Year == "2001"].copy()
data_05 = res[res.Year == "2005"].copy()
data_10 = res[res.Year == "2010"].copy()
data_15 = res[res.Year == "2015"].copy()
data_18 = res[res.Year == "2018"].copy()

In [ ]:
data_05.drop("Year", inplace = True, axis =1)
data_10.drop("Year", inplace = True, axis =1)
data_15.drop("Year", inplace = True, axis =1)
data_18.drop("Year", inplace = True, axis =1)

In [ ]:
temp1 = data_05.merge(data_10, on = "Name", how = "outer", suffixes = ["05", "10"])
temp2 = data_15.merge(data_18, on = "Name", how = "outer", suffixes = ["15", "18"])
fin  = temp1.merge(temp2, on = "Name", how = "outer")

### Converting Data types for numerical value

In [ ]:
fin.dtypes

In [ ]:
fin.Value05  = fin.Value05.str.replace(",","").astype(float)
fin.Value10  = fin.Value10.str.replace(",","").astype(float)
fin.Value15  = fin.Value15.str.replace(",","").astype(float)
fin.Value18  = fin.Value18.str.replace(",","").astype(float)
fin.Percent_value05 = fin.Percent_value05.astype(float)
fin.Percent_value10 = fin.Percent_value10.astype(float)
fin.Percent_value15 = fin.Percent_value15.astype(float)
fin.Percent_value18 = fin.Percent_value18.astype(float)

### Plotting yearwise transition for top few Country. For Percent_value, we are ignoring top few as they are city state which means 100% of polution will be in the capital city.

In [ ]:
data = fin[["Name", "Value05", "Value10", "Value15", "Value18"]].sort_values('Value18', ascending=False).iloc[0:15]
data.plot(kind="bar",x= "Name")
data2 = fin[["Name", "Percent_value05", "Percent_value10", "Percent_value15", "Percent_value18"]].sort_values('Percent_value15', ascending=False).iloc[0:15]
data2.plot(kind="bar", x= "Name")
data2 = fin[["Name", "Percent_value05", "Percent_value10", "Percent_value15", "Percent_value18"]].sort_values('Percent_value15', ascending=False).iloc[3:18]
data2.plot(kind="bar", x= "Name")
# help(data.plot())

# Conclusion

* Japan is most populated capital of world.
* New Delhi is fastest growing capital among top 10 and 2nd largest capital of world
* Macau , HongKong, Singapore 
* City of Kuwaits percent population of total country is shrinking. I oculd be because other places are getting populated